# 这个文件是下载并导入的文件

In [1]:
import GEOparse
import datetime

In [11]:
GEONAME = 'GSE63990'
# GSE63990  Profiling of bacterial respiratory infection, viral respiratory infection, and non-infectious illness
# GSE6269   Gene expression patterns in blood leukocytes discriminate patients with acute infections


In [12]:
gse = GEOparse.get_GEO(geo=GEONAME, destdir="./", silent=False)

15-Apr-2023 22:06:54 DEBUG utils - Directory ./ already exists. Skipping.
15-Apr-2023 22:06:54 INFO GEOparse - File already exist: using local version.
15-Apr-2023 22:06:54 INFO GEOparse - Parsing ./GSE63990_family.soft.gz: 
15-Apr-2023 22:06:54 DEBUG GEOparse - DATABASE: GeoMiame
15-Apr-2023 22:06:54 DEBUG GEOparse - SERIES: GSE63990
15-Apr-2023 22:06:54 DEBUG GEOparse - PLATFORM: GPL571
15-Apr-2023 22:06:55 DEBUG GEOparse - SAMPLE: GSM1561860
15-Apr-2023 22:06:55 DEBUG GEOparse - SAMPLE: GSM1561861
15-Apr-2023 22:06:55 DEBUG GEOparse - SAMPLE: GSM1561862
15-Apr-2023 22:06:55 DEBUG GEOparse - SAMPLE: GSM1561863
15-Apr-2023 22:06:55 DEBUG GEOparse - SAMPLE: GSM1561864
15-Apr-2023 22:06:56 DEBUG GEOparse - SAMPLE: GSM1561865
15-Apr-2023 22:06:56 DEBUG GEOparse - SAMPLE: GSM1561866
15-Apr-2023 22:06:56 DEBUG GEOparse - SAMPLE: GSM1561867
15-Apr-2023 22:06:56 DEBUG GEOparse - SAMPLE: GSM1561868
15-Apr-2023 22:06:56 DEBUG GEOparse - SAMPLE: GSM1561869
15-Apr-2023 22:06:56 DEBUG GEOparse - 

In [13]:
gse.get_metadata_attribute("title")

'Profiling of bacterial respiratory infection, viral respiratory infection, and non-infectious illness'

In [14]:
# 先把一个个样本的Value值提取出来，然后再转置，这样就可以得到一个样本对应的所有基因的表达值。
sample_value = gse.pivot_samples('VALUE').T
sample_value.tail()

ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Bs-thr-M_s_at,AFFX-r2-Ec-bioB-3_at,AFFX-r2-Ec-bioB-5_at,AFFX-r2-Ec-bioB-M_at,AFFX-r2-Ec-bioC-3_at,AFFX-r2-Ec-bioC-5_at,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at
name,,,,,,,,,,,,,,,,,,,,,
GSM1562135,7.997078,6.999220,12.230161,9.222677,4.046993,10.320181,5.947784,4.454022,10.917984,4.131798,...,4.322495,9.271240,9.991727,10.090862,11.671457,11.104189,13.377208,12.668151,14.386062,14.413862
GSM1562136,7.772143,7.555058,11.301208,8.430338,3.931747,9.506107,5.969773,4.304191,12.648006,4.227922,...,3.814574,9.169546,9.874713,9.970766,11.614028,11.000584,13.381485,12.673265,14.510159,14.461678
GSM1562137,7.836087,6.744497,11.414416,9.062577,3.973130,8.977779,5.958709,4.386142,11.428022,3.982706,...,4.220341,9.109918,9.854083,9.770358,11.510466,10.851155,13.301682,12.644916,14.493815,14.434904
GSM1562138,7.380660,7.190536,10.767835,8.874959,3.827331,8.425334,5.665389,4.344472,10.374727,3.817724,...,3.896055,9.096647,9.660840,9.764914,11.420875,10.804525,13.306708,12.686090,14.486557,14.465445
GSM1562139,7.914411,6.611748,9.930605,9.389512,4.514035,8.699409,6.353547,4.603126,11.575944,3.796684,...,4.238733,11.029648,11.785116,11.685551,12.899752,12.589470,14.021315,13.739892,14.664073,14.768721


In [20]:
value_desc = []
Value_Types = [
    'MAS5-calculated Signal intensity',
    'RMA normalized and Log2 transformed'
]
infection = []
n = 0
# value = [] we want 2-d dataframe
import pandas as pd
value = pd.DataFrame()
for i in gse.gsms:
    n += 1


    # 感染状态的处理
    characteristics_ch1 = gse.gsms[i].metadata['characteristics_ch1']   # the infection_status is in the characteristics_ch1
    infection_status = characteristics_ch1[-1].split(':')[1].strip()            # 但characteristics_ch1有的有两个值，有的有三个，取最后一个
    infection.append(infection_status)
    
    # 对数等的处理
    value_type = value_desc.append(gse.gsms[i].columns["description"][1])
    if gse.gsms[i].columns["description"][1] == 'RMA normalized and Log2 transformed':
        # 将sample_value 里第 n 行的每一个值转化为2的x次方
        value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
        # same line as above
    else:
        print(gse.gsms[i].columns["description"][1])
        print('Now just use the raw value')
        print('Samples: ', sample_value.iloc[n-1].head())
        print('-'*50)
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)

value = value.T

# merge the infection status and value
value.insert(0, 'infection_status', infection)

value.head()

ID_REF,infection_status,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,...,AFFX-r2-Bs-thr-M_s_at,AFFX-r2-Ec-bioB-3_at,AFFX-r2-Ec-bioB-5_at,AFFX-r2-Ec-bioB-M_at,AFFX-r2-Ec-bioC-3_at,AFFX-r2-Ec-bioC-5_at,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at
GSM1561860,non-infectious illness,107.026284,158.817338,2545.754898,186.244424,13.132469,643.402073,34.615676,17.490946,2929.019555,...,17.173226,422.464901,566.805882,481.918815,1421.477295,1302.020303,6250.362334,3836.600257,17740.725022,14961.589573
GSM1561861,non-infectious illness,221.020421,124.644738,2143.929143,324.223899,15.427804,1001.622909,49.833883,22.848206,5667.531661,...,17.962223,652.117965,800.852865,822.133404,2305.851262,2008.050881,8731.397156,5399.129367,19067.890695,17285.989865
GSM1561862,non-infectious illness,137.091142,115.353642,1741.194207,299.526479,13.588461,549.283988,47.869510,18.544460,3186.032084,...,17.282754,483.392556,566.237470,586.430031,1792.943487,1330.467773,7401.441597,4578.962962,18303.281696,15753.041045
GSM1561863,non-infectious illness,126.175162,98.119250,1316.526914,326.999326,12.738011,339.984798,51.808930,19.624606,5535.493091,...,17.651712,1275.694498,1781.291882,1547.788940,3845.464156,3549.057271,11770.475723,7924.052042,23370.081259,19856.779557
GSM1561864,bacterial,105.348086,260.359324,2930.300695,223.766173,12.436344,409.723526,35.071703,19.844859,581.984161,...,18.275624,404.702873,518.880900,477.676689,1299.615932,1186.047735,5968.686731,3780.851630,18440.196387,15535.017110


In [16]:
# Save the data and Show the first 5 rows
value.to_csv(GEONAME + '.csv', index=False)
value.head()

ID_REF,infection_status,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,...,AFFX-r2-Bs-thr-M_s_at,AFFX-r2-Ec-bioB-3_at,AFFX-r2-Ec-bioB-5_at,AFFX-r2-Ec-bioB-M_at,AFFX-r2-Ec-bioC-3_at,AFFX-r2-Ec-bioC-5_at,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at
GSM1561860,non-infectious illness,107.026284,158.817338,2545.754898,186.244424,13.132469,643.402073,34.615676,17.490946,2929.019555,...,17.173226,422.464901,566.805882,481.918815,1421.477295,1302.020303,6250.362334,3836.600257,17740.725022,14961.589573
GSM1561861,non-infectious illness,221.020421,124.644738,2143.929143,324.223899,15.427804,1001.622909,49.833883,22.848206,5667.531661,...,17.962223,652.117965,800.852865,822.133404,2305.851262,2008.050881,8731.397156,5399.129367,19067.890695,17285.989865
GSM1561862,non-infectious illness,137.091142,115.353642,1741.194207,299.526479,13.588461,549.283988,47.869510,18.544460,3186.032084,...,17.282754,483.392556,566.237470,586.430031,1792.943487,1330.467773,7401.441597,4578.962962,18303.281696,15753.041045
GSM1561863,non-infectious illness,126.175162,98.119250,1316.526914,326.999326,12.738011,339.984798,51.808930,19.624606,5535.493091,...,17.651712,1275.694498,1781.291882,1547.788940,3845.464156,3549.057271,11770.475723,7924.052042,23370.081259,19856.779557
GSM1561864,bacterial,105.348086,260.359324,2930.300695,223.766173,12.436344,409.723526,35.071703,19.844859,581.984161,...,18.275624,404.702873,518.880900,477.676689,1299.615932,1186.047735,5968.686731,3780.851630,18440.196387,15535.017110


## 不用了

不清楚什么时候再把这个id转化成通用的Gene Symbol。

可能在这里就转，也可以分析结束了再转。

大概需要10分钟

In [17]:
# GPL571 = gse.gpls['GPL571']
# n = 0
# for rows in sample_value:
#     Gene_Symbol = GPL571.table[GPL571.table['ID'] == rows]['Gene Symbol']
#     if Gene_Symbol.empty:
#         Gene_Symbol[0] = rows
#     sample_value.rename(columns={rows: Gene_Symbol.values[0]}, inplace=True)
#     n += 1
#     if n % 100 == 0:
#         print("Finished %d rows of %d at %s" % (n, sample_value.shape[1], datetime.datetime.now()))
# sample_value.tail()

In [18]:
# gse.gpls['GPL571'].table.columns # GPL571, 也就是这个GEO数据集的平台，的列。
# # 从这个平台可以直接读取Gene Symbol，Target Description等等有用的东西。

In [19]:
# gse.gpls['GPL571'].table.head() # 前5行